Nombre de los participantes Proyecto 
Juan José Naranjo Velásquez CC : 1039474689 
Alejandro Mora Suarez CC: 1128437044 
Kenneth David Leonel Triana CC: 1192817456

In [86]:
# importar librerias
import pandas as pd
import numpy as np

Obtener los datos y dejarlos en dataframes

In [87]:
#Extraer datos de calidad del aire en las estaciones
def get_data (url):
    return pd.read_csv(url)

cal_aire_ita = 'https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/28_itagui/estacion_data_calidadaire_28_20240301_20240331.csv'
cal_aire_cal = 'https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/69_Caldas/estacion_data_calidadaire_69_20240301_20240331.csv'
cal_aire_ara = 'https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/86_Aranjuez/estacion_data_calidadaire_86_20240301_20240331.csv'


cal_aire_ita = get_data(cal_aire_ita)
cal_aire_cal = get_data(cal_aire_cal)
cal_aire_ara = get_data(cal_aire_ara)

# Datos de temperatura,presion,humedad,precipitacion y vientos estacion itagui
temp_ita='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/28_itagui/Temperatura/estacion_data_temperatura_252__20240201_20240229.csv'
humedad_ita='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/28_itagui/humedad/estacion_data_humedad_252__20240201_20240229.csv'
presion_ita='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/28_itagui/presion/estacion_data_presion_252__20240201_20240229.csv'
precipitacion_ita='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/69_Caldas/precipitacion/estacion_data_precipitacion_105__20240201_20240229.csv'
vientos_ita='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/28_itagui/vientos/estacion_data_vientos_252__20240201_20240229.csv'

temp_ita=get_data(temp_ita)
humedad_ita=get_data(humedad_ita)
presion_ita=get_data(presion_ita)
precipitacion_ita=get_data(precipitacion_ita)
vientos_ita=get_data(vientos_ita)


# Datos de temperatura,presion,humedad y vientos estacion caldas

temp_cal='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/69_Caldas/Temperatura/estacion_data_temperatura_105__20240201_20240229.csv'
humedad_cal='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/69_Caldas/humedad/estacion_data_humedad_105__20240201_20240229.csv'
precipitacion_cal='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/69_Caldas/precipitacion/estacion_data_precipitacion_105__20240201_20240229.csv'
presion_cal='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/69_Caldas/presion/estacion_data_presion_105__20240201_20240229.csv'
vientos_cal='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/69_Caldas/vientos/estacion_data_vientos_105__20240201_20240229.csv'

temp_cal=get_data(temp_cal)
humedad_cal=get_data(humedad_cal)
precipitacion_cal=get_data(precipitacion_cal)
presion_cal=get_data(presion_cal)
vientos_cal=get_data(vientos_cal)


# Datos de temperatura,presion,humedad y vientos estacion aranjuez
temp_aranj='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/86_Aranjuez/temperatura/estacion_data_temperatura_68__20240201_20240229.csv'
humedad_aranj='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/86_Aranjuez/humedad/estacion_data_humedad_68__20240201_20240229.csv'
precipitacion_aranj='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/86_Aranjuez/precipitacion/estacion_data_precipitacion_68__20240201_20240229.csv'
presion_aranj='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/86_Aranjuez/presion/estacion_data_presion_68__20240201_20240229.csv'
vientos_aranj='https://raw.githubusercontent.com/kennethLeonel/Monografia-calidad-del-aire-valle-de-aburra/main/86_Aranjuez/vientos/estacion_data_vientos_68__20240201_20240229.csv'

temp_aranj=get_data(temp_aranj)
humedad_aranj=get_data(humedad_aranj)
precipitacion_aranj=get_data(precipitacion_aranj)
presion_aranj=get_data(presion_aranj)
vientos_aranj=get_data(vientos_aranj)

# nombres de caracteristicas en minusculas calidad aire
cal_aire_ita.columns = cal_aire_ita.columns.str.lower()
cal_aire_cal.columns = cal_aire_cal.columns.str.lower()
cal_aire_ara.columns = cal_aire_ara.columns.str.lower()

#nombres de caracteristicas en minusculas Itagui
temp_ita.columns = temp_ita.columns.str.lower()
humedad_ita.columns = humedad_ita.columns.str.lower()
presion_ita.columns = presion_ita.columns.str.lower()
vientos_ita.columns = vientos_ita.columns.str.lower()
precipitacion_ita.columns = precipitacion_ita.columns.str.lower()

#nombres de caracteristicas en minusculas Aranjuez
temp_aranj.columns = temp_aranj.columns.str.lower()
humedad_aranj.columns = humedad_aranj.columns.str.lower()
precipitacion_aranj.columns = precipitacion_aranj.columns.str.lower()
presion_aranj.columns = presion_aranj.columns.str.lower()
vientos_aranj.columns = vientos_aranj.columns.str.lower()

#nombres de caracteristicas en minusculas Caldas
temp_cal.columns = temp_cal.columns.str.lower()
humedad_cal.columns = humedad_cal.columns.str.lower()
precipitacion_cal.columns = precipitacion_cal.columns.str.lower()
presion_cal.columns = presion_cal.columns.str.lower()
vientos_cal.columns = vientos_cal.columns.str.lower()

Cambiar nombre columna calidad

In [97]:

def cambiarNombre(data,nombre ,nombreNuevo):
    """
    Función para cambiar nombre de columnas para uniformidad
    """
    # Cambiar nombre de columnas para mejor entendimiento
    data = data.rename(columns={nombre : nombreNuevo})
    return data

# Cambiar nombre de columnas para mejor entendimiento de los dataframes Temperatura Presion Humedad Itagui
temp_ita = cambiarNombre(temp_ita,'calidad', 'calidad_temperatura')
humedad_ita = cambiarNombre(humedad_ita,'calidad', 'calidad_humedad')
presion_ita = cambiarNombre(presion_ita,'calidad', 'calidad_presion')
vientos_ita = cambiarNombre(vientos_ita,'calidad', 'calidad_viento')
precipitacion_ita = cambiarNombre(precipitacion_ita,'calidad', 'calidad_precipitacion')

# Cambiar nombre de columnas para mejor entendimiento de los dataframes Temperatura Presion Humedad Caldas
temp_cal = cambiarNombre(temp_cal,'calidad', 'calidad_temperatura')
humedad_cal = cambiarNombre(humedad_cal,'calidad', 'calidad_humedad')
precipitacion_cal = cambiarNombre(precipitacion_cal,'calidad', 'calidad_precipitacion')
presion_cal = cambiarNombre(presion_cal,'calidad', 'calidad_presion')
vientos_cal = cambiarNombre(vientos_cal,'calidad', 'calidad_viento')

# Cambiar nombre de columnas para mejor entendimiento de los dataframes Temperatura Presion Humedad Aranjuez
temp_aranj = cambiarNombre(temp_aranj,'calidad', 'calidad_temperatura')
humedad_aranj = cambiarNombre(humedad_aranj,'calidad', 'calidad_humedad')
precipitacion_aranj = cambiarNombre(precipitacion_aranj,'calidad', 'calidad_precipitacion')
presion_aranj = cambiarNombre(presion_aranj,'calidad', 'calidad_presion')
vientos_aranj = cambiarNombre(vientos_aranj,'calidad', 'calidad_viento')


    


Filtrar el dataframe calidad aire para tomar las columnas objetivo del proyecto

In [102]:
# cal_aire_ara

def filtro_data_peque(data):
    """"
    Función para tomar columnas de estudio en archivos de calidad del aire en las diferentes estaciones

    In[0]: dataset
    Out[0]: dataset filtrado hasta columna de estudio
    """
    # Filtrar los datos segun el indice
    tamColumnas = data.shape
    if (tamColumnas[1]>6):
        datosFiltrados = data.iloc[:,:4]
    return datosFiltrados
        
def filtro_data(data,nombreColumna,condicion):
    """"
    Función para tomar calidad de datos en tiempo real establecidos desde datos crudos

    In[0]: dataset, 'calidad_pm25', 1
    Out[0]: dataset filtrado con datos en calidad
    """
    # Filtrar los datos segun el indice y la condicion
    tamColumnas = data.shape
    if (tamColumnas[1]>6):
        datosFiltrados = data.iloc[:,:4]
        return datosFiltrados[datosFiltrados.loc[:, nombreColumna] == condicion]
    else:
       return data.loc[data.loc[:, nombreColumna] == condicion]

#Se tiene el filtrado estación Itagui    
dataCalidadFiltradaItagui=filtro_data_peque(cal_aire_ita)

#Se tiene el filtrado estación Caldas 
dataCalidadFiltradaCaldas = filtro_data_peque(cal_aire_cal)

#Se tiene el filtrado estación Aranjuez
dataCalidadFiltradaAranjuez = filtro_data_peque(cal_aire_ara)

#Visualizar catos calidad aire itagui
dataCalidadFiltradaItagui




,fecha_hora,codigoserial,pm25,calidad_pm25
0,2024-03-01 00:00:00,28,59.0,1.0
1,2024-03-01 01:00:00,28,53.0,1.0
2,2024-03-01 02:00:00,28,59.0,1.0
3,2024-03-01 03:00:00,28,45.0,1.0
4,2024-03-01 04:00:00,28,49.0,1.0
...,...,...,...,...
739,2024-03-31 19:00:00,28,26.0,1.0
740,2024-03-31 20:00:00,28,41.0,1.0
741,2024-03-31 21:00:00,28,35.0,1.0
742,2024-03-31 22:00:00,28,38.0,1.0


Sacar promedio de los dataframes (Temperatura , precipitación, Vientos, Humedad, Presion) de las diferentes estaciones

In [91]:
from scipy import stats
def agregarColumnas(data):
    """"
    Función para agregar columnas 
    In[0]: dataset
    Out[0]: dataset con columnas día , mes
    """
    dataTemporal = data.copy()
    dataTemporal['fecha'] = pd.to_datetime(dataTemporal['fecha_hora']).dt.date
    dataTemporal['dia']= pd.to_datetime(dataTemporal['fecha_hora']).dt.day
    dataTemporal['mes']= pd.to_datetime(dataTemporal['fecha_hora']).dt.month
    return dataTemporal


def sacarPromedio(data,nombreColumna,calidad,revision):
    """"
    Función para calcular el promedio de todas las columnas de un dataframe
    In[0]: dataset
    Out[0]: promedio de cada columna
    """
    #Llamamos a la funcion agregarColumnas
    dataNuevo = agregarColumnas(data)
    #return dataNuevo
    #se saca el promedio de temperatura de la columna Temperatura
    # registroPromedio = data.groupby(['mes','dia'])[['Temperatura']].reset_index()
   

    if (revision ==1):
        registroPromedio = dataNuevo.groupby(['mes','dia']).agg({
            f'{nombreColumna}':'mean',
            f'{calidad}':lambda x: stats.mode(x)[0][0]
            
        }).reset_index()
        return registroPromedio
    elif(revision ==2):
        registroPromedio = dataNuevo.groupby(['mes','dia']).agg({
            f'{nombreColumna}':'mean',
            f'{calidad}':lambda x: stats.mode(x)[0][0],
            'codigoserial':lambda x: stats.mode(x)[0][0]
            
        }).reset_index()
        return registroPromedio
    elif(revision == 3):
        registroPromedio = dataNuevo.groupby(['mes','dia']).agg({
            f'{nombreColumna}':'mean',
            f'{calidad}':lambda x: stats.mode(x)[0][0],
            'velocidad_max':'mean',
            'direccion_prom':'mean',
            'direccion_max':'mean' 
        }).reset_index()
        return registroPromedio




In [109]:
#Sacar promedio Itagui
temperaturaItaguiPromedio = sacarPromedio(temp_ita, 'temperatura', 'calidad_temperatura',1)
humedadItaguiPromedio = sacarPromedio(humedad_ita, 'humedad','calidad_humedad',1)
presionItaguiPromedio = sacarPromedio(presion_ita, 'presion','calidad_presion',1)
precipitacionItaguiPromedio = sacarPromedio(precipitacion_ita, 'p1','calidad_precipitacion',1)
vientosItaguiPromedio = sacarPromedio(vientos_ita,'velocidad_prom','calidad_viento',3) 

#Sacar promedio Caldas
temperaturaCaldasPromedio = sacarPromedio(temp_cal, 'temperatura', 'calidad_temperatura',1)
humedadCaldasPromedio = sacarPromedio(humedad_cal,'humedad','calidad_humedad',1)
precipitacionCaldasPromedio = sacarPromedio(precipitacion_cal, 'p1','calidad_precipitacion',1)
presionCaldasPromedio = sacarPromedio(presion_cal, 'presion','calidad_presion',1)
vientosCaldasPromedio = sacarPromedio(vientos_cal,'velocidad_prom','calidad_viento',3) 

#Sacar promedio Aranjuez
temperaturaAranjuezPromedio = sacarPromedio(temp_aranj, 'temperatura', 'calidad_temperatura',1)
humedadAranjuezPromedio = sacarPromedio(humedad_aranj,'humedad','calidad_humedad',1)
precipitacionAranjuezPromedio = sacarPromedio(precipitacion_aranj, 'p1','calidad_precipitacion',1)
presionAranjuezPromedio = sacarPromedio(presion_aranj, 'presion','calidad_presion',1)
vientosAranjuezPromedio = sacarPromedio(vientos_aranj,'velocidad_prom','calidad_viento',3) 



C:\Users\USER\AppData\Local\Temp\ipykernel_11304\2607153054.py:31: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  f'{calidad}':lambda x: stats.mode(x)[0][0]
C:\Users\USER\AppData\Local\Temp\ipykernel_11304\2607153054.py:31: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warnin

Al dataframe dataCalidadFiltrada se necesita sacar el promedio por dia para después concatanar las demas variables con el dataframe

In [114]:
calidadAireItagui = sacarPromedio(dataCalidadFiltradaItagui, 'pm25', 'calidad_pm25',2)
calidadAireCaldas = sacarPromedio(dataCalidadFiltradaCaldas, 'pm25', 'calidad_pm25',2)
calidadAireAranjuez = sacarPromedio(dataCalidadFiltradaAranjuez, 'pm25', 'calidad_pm25',2)
calidadAireItagui.iloc[:3]

C:\Users\USER\AppData\Local\Temp\ipykernel_11304\2607153054.py:38: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  f'{calidad}':lambda x: stats.mode(x)[0][0],
C:\Users\USER\AppData\Local\Temp\ipykernel_11304\2607153054.py:39: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warni

,mes,dia,pm25,calidad_pm25,codigoserial
0,3,1,42.208333,1.0,28
1,3,2,42.500000,1.0,28
2,3,3,30.666667,1.0,28


Unión de dataFrames

In [96]:
#Unir los datframes
def unirDatos(data1, data2):
    """"
    Función para unir dos dataframes
    In[0]: dataset1, dataset2
    Out[0]: dataset unido
    """
    return pd.merge(data1,data2,on=['mes','dia'],how='inner')

#Unir los dataframes de la estación Itagui
calidadAireItaguiUnido = unirDatos(calidadAireItagui,temperaturaItaguiPromedio)
calidadAireItaguiUnido = unirDatos(calidadAireItaguiUnido,humedadItaguiPromedio)
calidadAireItaguiUnido = unirDatos(calidadAireItaguiUnido,presionItaguiPromedio)
calidadAireItaguiUnido = unirDatos(calidadAireItaguiUnido,precipitacionItaguiPromedio)
calidadAireItaguiUnido = unirDatos(calidadAireItaguiUnido,vientosItaguiPromedio)

#Unir los dataframes de la estación Caldas

calidadAireCaldasUnido = unirDatos(calidadAireCaldas,temperaturaCaldasPromedio)
calidadAireCaldasUnido = unirDatos(calidadAireCaldasUnido,humedadCaldasPromedio)
calidadAireCaldasUnido = unirDatos(calidadAireCaldasUnido,precipitacionCaldasPromedio)
calidadAireCaldasUnido = unirDatos(calidadAireCaldasUnido,presionCaldasPromedio)
calidadAireCaldasUnido = unirDatos(calidadAireCaldasUnido,vientosCaldasPromedio)

#Unir los dataframes de la estación Aranjuez

calidadAireAranjuezUnido = unirDatos(calidadAireAranjuez,temperaturaAranjuezPromedio)
calidadAireAranjuezUnido = unirDatos(calidadAireAranjuezUnido,humedadAranjuezPromedio)
calidadAireAranjuezUnido = unirDatos(calidadAireAranjuezUnido,precipitacionAranjuezPromedio)
calidadAireAranjuezUnido = unirDatos(calidadAireAranjuezUnido,presionAranjuezPromedio)
calidadAireAranjuezUnido = unirDatos(calidadAireAranjuezUnido,vientosAranjuezPromedio)

calidadAireItaguiUnido.iloc[:3]

,mes,dia,pm25,calidad_pm25,codigoserial,temperatura,calidad_temperatura,humedad,calidad_humedad,presion,calidad_presion,p1,calidad_precipitacion,velocidad_prom,calidad_viento,velocidad_max,direccion_prom,direccion_max
0,3,1,42.208333,1.0,28,23.662555,155,72.425576,155,0.0,155,0.000021,1,2.092007,155,3.097708,121.080556,135.025000
1,3,2,42.500000,1.0,28,22.775778,155,72.182312,155,0.0,155,0.000007,1,2.082181,155,3.207917,139.937500,157.909722
2,3,3,30.666667,1.0,28,21.846292,155,77.600868,155,0.0,155,0.012653,1,2.192493,155,3.410556,151.859028,166.261806


Hacer el mismo proceso con la estacion Caldas

In [116]:
def ConcatenadoRegistros(*data):
    """"
    Función para concatenar tres dataframes
    In[0]: (dataset1, dataset2, dataset3, ..., datasetn)
    Out[0]: dataset concatenado
    """
    for i in data:
        return pd.concat([i], axis=0)

calidadAireCaldas = ConcatenadoRegistros(temp_ita, temp_cal, temp_aranj)
calidadAireCaldas

,fecha_hora,temperatura,calidad_temperatura
0,2024-02-01 00:00:00,21.500000,155
1,2024-02-01 00:01:00,21.500000,155
2,2024-02-01 00:02:00,21.500000,155
3,2024-02-01 00:03:00,21.500000,155
4,2024-02-01 00:04:00,21.459999,155
...,...,...,...
86395,2024-03-31 23:55:00,19.600000,155
86396,2024-03-31 23:56:00,19.660000,155
86397,2024-03-31 23:57:00,19.709999,155
86398,2024-03-31 23:58:00,19.760000,155
